In [1]:
'''
MS4: ArcPy functions

Series of data preparation scripts that are done in the Esri environment. These prepare 
data that are used in the rest of the analysis. In future iterations as input data layers are updated,
these may need to be rerun (one at a time, likely). Once they are run, the updates need to be reflected
in make_dataset.py and then the other scripts can be run as usual.

Created May 22, 2023 by RBowers

'''

import arcpy 
from arcpy.sa import *
from arcpy.ia import *
from arcpy.da import *
import numpy as np
import os

## Tesselation
This script creates a 400 sqm tesselation (hexagon fishnet) that covers each MAPC municipality. Likely won't need to be run again for this analysis. Used in 01-initializing-project.ipynb to create one of the base analysis geographies

In [49]:
#create fishnets over each town - takes a while to do all munis! 

out_gdb = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Fishnets.gdb"
in_polys = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Tool_Preprocessing.gdb\\MAPCTowns"
#arcpy.env.overwriteOutput = True #allow overwriting the output (turn on if you decide to overwrite)

# spatial reference
arcpy.env.outputCoordinateSystem = arcpy.Describe(in_polys).spatialReference

# Loop thru rows of input polygons
with arcpy.da.SearchCursor(in_polys, [['SHAPE@', 'municipal']]) as rows:
    for row in rows:
        ext = row[0].extent
        # Create fishnet
        out = os.path.join(out_gdb, ('tess_' + row[1]).format(row[1]))
        if arcpy.Exists(out):#if you want to replace existing tesselations, remove the if/else and just do the generate tesselation bit with env.overwriteoutput as True
            pass
        else:
            arcpy.management.GenerateTessellation(out, ext,  Size='400 SquareMeters')




## Imperviousness and Structures

Create a feature class for impervious surfaces MINUS building structure boundaries. As MassGIS structures are updated in the future, this may also need to be updated.

In [8]:
#erase building structures from the impervious surfaces data
in_features = r'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\MS4_Model.gdb\imperv_simplify_all_mapc'
erase_features = r'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\MS4_Model.gdb\mapc_structures' #this was clipped ahead of time to the mapc boundary
out_feature_class = r"K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\MS4_Model.gdb\imperv_erase_structures"

#set environment to allow for overwriting and to ensure the output is in the correct crs
arcpy.env.overwriteOutput = True #allow overwriting the output
arcpy.env.outputCoordinateSystem = arcpy.Describe(in_features).spatialReference

#perform erase
arcpy.analysis.PairwiseErase(in_features, 
                            erase_features, 
                            out_feature_class
                            )

In [5]:
heat_fp = 'K:\\DataServices\\Projects\\Current_Projects\\Climate_Change\\MVP_MMC_Heat_MVP\\00 Task 2 Deliverables\\2.1 Attachments\\00 Uploaded to Sharepoint\\Shapefile_LSTIndex\\LSTindex.tif'
lclu = "K:\\DataServices\\Datasets\\Zoning and Land Use\\Spatial\\lclu_gdb\\MA_LCLU2016.gdb\\LandCoverLandUse_2016"
soils = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Tool_Preprocessing.gdb\\SOILS_MUPOLYGON_TOP20_MAPCclip"

output_crs = "PROJCS[\"NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001\",GEOGCS[\"GCS_North_American_1983\",DATUM[\"D_North_American_1983\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Conformal_Conic\"],PARAMETER[\"False_Easting\",200000.0],PARAMETER[\"False_Northing\",750000.0],PARAMETER[\"Central_Meridian\",-71.5],PARAMETER[\"Standard_Parallel_1\",41.71666666666667],PARAMETER[\"Standard_Parallel_2\",42.68333333333333],PARAMETER[\"Latitude_Of_Origin\",41.0],UNIT[\"Meter\",1.0]]"

In [12]:
LSTindex_tif = arcpy.Raster(heat_fp)


arcpy.env.extent = LSTindex_tif
arcpy.env.snapRaster = LSTindex_tif
arcpy.env.cellSize = LSTindex_tif
arcpy.env.mask = LSTindex_tif

In [10]:
def AutoName(table): # function that automatically names a feature class or raster
    # Adapted from MAPC's stormwater toolkit script at https://github.com/MAPC/stormwater-toolkit/blob/master/Burn_Raster_Script.py
    checktable = arcpy.Exists(table) # checks to see if the raster already exists
    count = 2
    newname = table

    while checktable == True: # if the raster already exists, adds a suffix to the end and checks again
        newname = table + str(count)
        count += 1
        checktable = arcpy.Exists(newname)

    return newname

In [13]:
def geotiff_convert(inname,                 # Name of input shapefile
                    outfeaturename,         # Desired name of converted raster
                    fieldname,              # Field value on which to base raster value
                    workspace,              # Workspace in which to place resulting raster
                    inputform = 'Polygon'): # set to 'Point' if working with point type shapefile
    
    # Check for geometry problems
    outtable = AutoName('geomtable')
    arcpy.CheckGeometry_management(inname, outtable)
    
    #Repair geometry problems
    arcpy.RepairGeometry_management(inname)
    
    # Convert to raster form
    if inputform == 'Polygon':
        outname = AutoName(outfeaturename)
        arcpy.PolygonToRaster_conversion(inname, fieldname, outname, cell_assignment = 'MAXIMUM_AREA')
        
    elif inputform == 'Point':
        outname = AutoName(outfeaturename)
        arcpy.PointToRaster_conversion(inname, fieldname, outname, cell_assignment = 'MOST_FREQUENT')
        
    else:
        print('Incorrect input form')
        
    # Convert Raster to geoTIFF
    arcpy.RasterToOtherFormat_conversion(outname, workspace, 'TIFF')
    
    # Build pyramids
    arcpy.BuildPyramids_management(outname)
    
    # Calculate statistics
    arcpy.CalculateStatistics_management(outname)
    
    return(outname)